In [ ]:
import wheel
import numpy as np
import sklearn as skl
from mat4py import loadmat
import scipy.io
import csv
import struct
import h5py
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps
#from mne_bids import BIDSPath, read_raw_bids
from scipy import stats
from scipy.signal import butter, filtfilt,welch
from scipy.stats import zscore 
#from mne.time_frequency import tfr_multitaper
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk
import seaborn as sns
import statsmodels.api as sm
from tqdm import tqdm

#import mne
#from mne.viz import plot_alignment, snapshot_brain_montage
#import mne_bids
#from mne.channels import make_standard_montage

from sklearn.model_selection import train_test_split
import pickle
import os
import dask
import dask.dataframe as dd
import statsmodels.api as sm


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colormaps
from mne_bids import BIDSPath, read_raw_bids
from scipy.signal import butter, filtfilt
from mne.time_frequency import tfr_multitaper
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import tkinter as tk



import mne
from mne.viz import plot_alignment, snapshot_brain_montage
import mne_bids
from mne.channels import make_standard_montage

In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('once')

In [ ]:
"""This file contains the tools neccesary to work with GPT-2-XL, GloVe, and BERT embeddings of your choosing and run ridge regressions 
with subject level and summary graphs. It also runs a GLM model for comparison. This file should only be run after 1)Running pooling file 2) Runnin the data prep file.
"""

In [ ]:
#The following three files are produced after 1)The pooling script 2) The dataprep script<- !!

In [ ]:
GS = pd.read_csv('./GS_corr.csv.gz')

In [ ]:
GNS = pd.read_csv('./GNS_corr.csv.gz')

In [ ]:
NGNS = pd.read_csv('./NGNS_corr.csv.gz')

In [ ]:
GNS.columns

In [ ]:
GS_new = GS
GNS_new = GNS
NGNS_new = NGNS

In [ ]:
GS_new

In [ ]:
#Import in sentence data

In [ ]:
sentence_data = pd.read_csv('sentence_data.csv.gz', compression = 'gzip')

sentence_data.dropna(how = 'any', inplace = True)
#Remove rejected subjects 
sentence_data = sentence_data[(sentence_data['sub_id'] != 3) & (sentence_data['sub_id'] != 17) & (sentence_data['sub_id'] != 1)]

In [ ]:
#Start modeling how to do GLM for each sub 

In [ ]:
#Start with subject_14

In [ ]:
sentence_data.drop_duplicates(subset = ['w1','w2','w3','w4','modality','sentenceType', 'sub_id'], inplace = True) #Ensure no repeat sentence data

In [ ]:
sentence_data

In [ ]:
#Handle numbering the electrodes per subject 

In [ ]:
subjects = list(set(sentence_data['sub_id']))
subjects

In [ ]:
GS_new

In [ ]:
elec_final = pd.concat([GS_new, GNS_new, NGNS_new])

In [ ]:
elec_final

In [ ]:
elec_final.index = elec_final['Unnamed: 0']

In [ ]:
elec_final.index

In [ ]:
#Creating a column to nicely aggregate participants and their electrodes
elec_final['elec_source'] = "P" + elec_final['sub_id'].astype(str) + '_' +elec_final.index.astype(str)


In [ ]:
elec_final['elec_source']

In [ ]:
elec_final.rename(columns = {"type": "sentenceType"}, inplace=True)

In [ ]:
elec_final.columns

In [ ]:
elec_final['sentenceType'].value_counts()

In [ ]:
#Drop flat electrodes
start_col = 3
end_col = 152

# Ensure that end_col does not exceed the number of columns in df
end_col = min(end_col, len(elec_final.columns) - 1)

# Check and drop rows where all values in the specified columns are zero
elec_final = elec_final[(elec_final.iloc[:, start_col:end_col + 1] != 0).any(axis=1)]

In [ ]:
elec_final.isna().sum()

In [ ]:
sentence_data.dropna(how = 'any',inplace= True)
sentence_data['sentenceType'].value_counts()

In [ ]:
elec_final = elec_final.sort_values(by='sub_id')

In [ ]:
electrodes_per = elec_final[elec_final['sentenceType'] == 'GS']['sub_id'].value_counts()

In [ ]:
electrodes_per_df = electrodes_per.reset_index()
electrodes_per_df.columns = ['participant', 'electrode_count']

# Sort by participant number
electrodes_per_df = electrodes_per_df.sort_values(by='participant')

# Plot histogram
plt.figure(figsize=(10, 6))
plt.bar(electrodes_per_df['participant'], electrodes_per_df['electrode_count'], color='skyblue')
plt.xlabel('Participant Number')
plt.ylabel('Number of Electrodes')
plt.title('Count of SEEG Electrodes per Participant')
plt.xticks(electrodes_per_df['participant'])


# Display the plot
plt.show()

In [ ]:
elec_final.columns

In [ ]:
sentence_data.reset_index(inplace = True)

In [ ]:
elec_final.columns[1:152]

In [1]:
#GLM code
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

def prepare_data(df, numeric_columns):
    # Convert numpy arrays to float arrays
    for col in numeric_columns:
        if isinstance(df[col].iloc[0], np.ndarray):
            df[col] = df[col].apply(lambda x: x.astype(float))
    
    # Drop rows with any NaN values
    df_clean = df.dropna()
    
    if len(df_clean) < len(df):
        print(f"Dropped {len(df) - len(df_clean)} rows with NaN values")
    
    return df_clean

# Merge the sentence and electrode data on 'sub_id' and 'sentenceType'
merged_df = pd.merge(sentence_data, elec_final, on=['sub_id', 'sentenceType'])

# Define numeric columns
numeric_columns = [str(i) for i in range(151)]  # Assuming columns for gamma power are named '0' to '150'

# Prepare the data
merged_df = prepare_data(merged_df, numeric_columns)

# Create results DataFrame with electrode names as index
electrode_names = merged_df['elec_source'].unique()
results = pd.DataFrame(index=electrode_names, columns=['GS_MSE', 'GS_R2', 'GS_p_value', 'GNS_MSE', 'GNS_R2', 'GNS_p_value', 'NGNS_MSE', 'NGNS_R2', 'NGNS_p_value'])
results.index.name = 'Electrode'

# Group the data by electrode
electrode_groups = merged_df.groupby('elec_source')

for electrode, electrode_df in electrode_groups:
    for condition in ['GS', 'GNS', 'NGNS']:
        # Filter DataFrame by condition for the current electrode
        condition_data = electrode_df[electrode_df['sentenceType'] == condition]
        if len(condition_data) == 0:
            print(f"Warning: No data for {condition} in electrode {electrode}")
            continue
        
        # Calculate mean gamma power per sample
        y = np.array([arr.mean() for arr in condition_data[numeric_columns].values])
        
        # Create dummy variables for categorical predictors
        X = pd.get_dummies(condition_data[['sentenceType', 'modality']], drop_first=True)
        
        # Add constant term
        X = sm.add_constant(X)
        
        try:
            # Perform GLM using statsmodels
            model = sm.OLS(y, X).fit()
            y_pred = model.predict(X)
            
            # Calculate Mean Squared Error and R-squared
            mse = mean_squared_error(y, y_pred)
            r2 = r2_score(y, y_pred)
            
            # Store results and p-value
            results.loc[electrode, f"{condition}_MSE"] = mse
            results.loc[electrode, f"{condition}_R2"] = r2
            results.loc[electrode, f"{condition}_p_value"] = model.pvalues[1] if len(model.pvalues) > 1 else np.nan
        except Exception as e:
            print(f"Error in electrode {electrode}, condition {condition}: {str(e)}")
            results.loc[electrode, [f"{condition}_MSE", f"{condition}_R2", f"{condition}_p_value"]] = np.nan

# Print summary statistics
print("\nSummary Statistics:")
print(results.describe())

# Print detailed results for a few electrodes
print("\nDetailed results for first 5 electrodes:")
print(results.head())

# Print data types of columns
print("\nData types of columns:")
print(merged_df.dtypes)

# Print sample of data
print("\nSample of data (first row, first 5 numeric columns):")
for col in numeric_columns[:5]:
    print(f"{col}: {merged_df[col].iloc[0][:5]}")  # Print first 5 elements of the array


KeyboardInterrupt



In [ ]:
results

In [ ]:
results_df = results_df[(results_df['R-squared'] > -1) & (results_df['R-squared'] < 1)]

In [ ]:
results_df

In [ ]:
from scipy.stats import zscore

# Calculate z-scores for R-squared values
results_df['z-score'] = zscore(results_df['R-squared'])

# Define outlier criteria (e.g., |z-score| > 3)
filtered_results_df = results_df[results_df['z-score'].abs() <= 3]

# Drop the z-score column as it's no longer needed
filtered_results_df = filtered_results_df.drop(columns=['z-score'])

# Save the filtered results to a new CSV
filtered_results_df.to_csv('filtered_glm_results_per_electrode.csv', index=False)

# Print the number of electrodes before and after filtering
print(f"Number of electrodes before filtering: {len(results_df)}")
print(f"Number of electrodes after filtering: {len(filtered_results_df)}")

In [ ]:
filtered_results_df[filtered_results_df['R-squared'] < 0]

In [ ]:
results_df['R-squared'].value_counts()

In [ ]:
filtered_results_df.to_csv('/Users/klab/Total_Sub_GLM.csv.gz', compression = 'gzip')

In [ ]:
results_df['R-squared'].value_counts()

In [ ]:
results_df

In [ ]:
results_df['R-squared'].max()

In [ ]:


# Plot R-squared values for each electrode
plt.figure(figsize=(14, 8))
plt.bar(results_df['Electrode'], results_df['R-squared'], color='skyblue')
plt.title('Subject 14 GLM R^2 per Electrode')
plt.xlabel('Electrode')
plt.ylabel('R-squared')
plt.xticks(rotation=90)  # Rotate electrode labels for better readability
plt.grid(axis='y')
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.savefig('r_squared_per_electrode_matplotlib.png')
plt.show()
plt.close()





In [ ]:
#Drop flat electrodes
start_col = 0
end_col = 2

# Ensure that end_col does not exceed the number of columns in df
end_col = min(end_col, len(coefficients_a_v.columns) - 1)

# Check and drop rows where all values in the specified columns are zero
coefficients_a_v = coefficients_a_v[(coefficients_a_v.iloc[:, start_col:end_col + 1] != 0).any(axis=1)]

In [ ]:
valid_elecs = results_df[(results_df['R-squared'] <= 1) & (results_df['R-squared'] >= -1)]

coefficients = pd.DataFrame(list(valid_elecs['P-values']))

coefficients.dropna(how = 'any', inplace = True)

#Drop flat electrodes
start_col = 0
end_col = 5

# Ensure that end_col does not exceed the number of columns in df
end_col = min(end_col, len(coefficients.columns) - 1)

# Check and drop rows where all values in the specified columns are zero
coefficients = coefficients[(coefficients.iloc[:, start_col:end_col + 1] != 0).any(axis=1)]

In [ ]:
coefficients_a_v = coefficients[['modality_a','modality_v']]

sig_a = coefficients_a_v['modality_a']
sig_v = coefficients_a_v['modality_v']

In [ ]:
coefficients_a

In [ ]:
del sig_a[92]
del sig_v[92]

In [ ]:
sig_a = dict(sig_a)

In [ ]:
sig_v = dict(sig_v)

In [ ]:
sig_a

In [ ]:
sig_GS = coefficients[coefficients['sentenceType_GS'] < 0.05]['sentenceType_GS']

In [ ]:
sig_GS = {92: 1.419178e-90}

In [ ]:
sig_NGNS = {92: 3.701506e-244}

In [ ]:
sig_NGNS

In [ ]:
results_df['P-values'][0].keys()

In [ ]:
sig_a = coefficients[coefficients['modality_a'] < 0.05]['modality_a']

In [ ]:
sig_a

In [ ]:
sig_v = coefficients[coefficients['modality_v'] < 0.05]['modality_v']

In [ ]:
sig_v

In [ ]:
sig_v

In [ ]:
sig_v = dict(sig_v)

In [ ]:
sig_GS

In [ ]:
sig_NGNS

In [ ]:
df_merged

In [ ]:



# Convert dictionaries to DataFrames
df_a = pd.DataFrame(sig_a.items(), columns=['Electrode', 'p_val_audio'])
df_v = pd.DataFrame(sig_v.items(), columns=['Electrode', 'p_val_visual'])

# Merge DataFrames on the 'Electrode' column
df_merged = pd.merge(df_a, df_v, on='Electrode', how='outer')

# Replace NaN values with a higher p-value (e.g., 1) to indicate non-significance
df_merged.fillna(1, inplace=True)

# Apply -log10 transformation
df_merged['neg_log10_p_val_audio'] = -np.log10(df_merged['p_val_audio'])
df_merged['neg_log10_p_val_visual'] = -np.log10(df_merged['p_val_visual'])

# Function to determine significance level and return corresponding asterisks
def significance_asterisks(p_val):
    if p_val < 0.001:
        return '***'
    elif p_val < 0.01:
        return '**'
    elif p_val < 0.05:
        return '*'
    else:
        return ''

# Add asterisks based on significance
df_merged['asterisks_audio'] = df_merged['p_val_audio'].apply(significance_asterisks)
df_merged['asterisks_visual'] = df_merged['p_val_visual'].apply(significance_asterisks)

# Plotting the grouped bar chart with log-transformed p-values
x = np.arange(len(df_merged))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, df_merged['neg_log10_p_val_audio'], width, label='Audio')
bars2 = ax.bar(x + width/2, df_merged['neg_log10_p_val_visual'], width, label='Visual')

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax.set_xlabel('Electrode')
ax.set_ylabel('-log10(p-value)')
ax.set_title('Significant Audiovisual Electrodes Subject 14')
ax.set_xticks(x)
ax.set_xticklabels(df_merged['Electrode'])
ax.legend()

# Function to add asterisks above the bars
def add_asterisks(bars, asterisks):
    for bar, asterisk in zip(bars, asterisks):
        height = bar.get_height()
        if asterisk:
            ax.annotate(asterisk,
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 2),  # 2 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom',
                        fontsize=10)  # Font size for asterisks

add_asterisks(bars1, df_merged['asterisks_audio'])
add_asterisks(bars2, df_merged['asterisks_visual'])

plt.show()





In [ ]:



# Convert dictionaries to DataFrames
df_g = pd.DataFrame(sig_GS.items(), columns=['Electrode', 'p_val_GS'])
df_n = pd.DataFrame(sig_NGNS.items(), columns=['Electrode', 'p_val_NGNS'])

# Merge DataFrames on the 'Electrode' column
df_merged = pd.merge(df_g, df_n, on='Electrode', how='outer')

# Replace NaN values with a higher p-value (e.g., 1) to indicate non-significance
df_merged.fillna(1, inplace=True)

# Apply -log10 transformation
df_merged['neg_log10_p_val_GS'] = -np.log10(df_merged['p_val_GS'])
df_merged['neg_log10_p_val_NGNS'] = -np.log10(df_merged['p_val_NGNS'])

# Function to determine significance level and return corresponding asterisks
def significance_asterisks(p_val):
    if p_val < 0.001:
        return '***'
    elif p_val < 0.01:
        return '**'
    elif p_val < 0.05:
        return '*'
    else:
        return ''

# Add asterisks based on significance
df_merged['asterisks_GS'] = df_merged['p_val_GS'].apply(significance_asterisks)
df_merged['asterisks_NGNS'] = df_merged['p_val_NGNS'].apply(significance_asterisks)

# Plotting the grouped bar chart with log-transformed p-values
x = np.arange(len(df_merged))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, df_merged['neg_log10_p_val_GS'], width, label='GS', color = "purple" )
bars2 = ax.bar(x + width/2, df_merged['neg_log10_p_val_NGNS'], width, label='NGNS', color = "yellow")

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax.set_xlabel('Electrode')
ax.set_ylabel('-log10(p-value)')
ax.set_title('Significant Grammatical Electrode 92 Subject 14')
ax.set_xticks(x)
ax.set_xticklabels(df_merged['Electrode'])
ax.legend()

# Function to add asterisks above the bars
def add_asterisks(bars, asterisks):
    for bar, asterisk in zip(bars, asterisks):
        height = bar.get_height()
        if asterisk:
            ax.annotate(asterisk,
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 2),  # 2 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom',
                        fontsize=10)  # Font size for asterisks

add_asterisks(bars1, df_merged['asterisks_GS'])
add_asterisks(bars2, df_merged['asterisks_NGNS'])

plt.show()





In [ ]:
curr_vals = dict(GS_new['sub_id'].value_counts())

In [ ]:
old_vals = dict(GS['sub_id'].value_counts())

In [ ]:
elec_count = total_elec['sub_id'].value_counts()

In [ ]:
#Do some language electrode analysis

In [ ]:
#Do per condition

In [ ]:
plot_prac = GS_new[GS_new['sub_id'] == 14]

In [ ]:
plot_prac.drop(['sub_id', 'type'], axis = 1, inplace = True)

In [ ]:
plot_prac.reset_index(inplace = True)

In [ ]:
plot_prac.index = plot_prac.index+1 #This needs to be done for all 

In [ ]:
import tkinter as tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scipy.stats import ttest_ind
import mne

# Assuming plot_prac is your DataFrame
# It should have electrodes as columns and time points as rows

# Set up parameters
sfreq = 1000  # Sampling frequency in Hz
total_duration = 6600  # Total duration of the experiment in milliseconds

# Create MNE info object
ch_names = plot_prac.columns.tolist()
info = mne.create_info(ch_names=ch_names, sfreq=sfreq)

# Convert DataFrame to numpy array
data = plot_prac.values.T  # Transpose to have channels as rows

# Create Raw object
raw = mne.io.RawArray(data, info)

# Define periods
periods = {
    "Baseline": (0, 600),
    "W1": (600, 1475),
    "W2": (1475, 2350),
    "W3": (2350, 3225),
    "W4": (3225, 4100),
    "Consolidate": (4100, 5100),
    "Response": (5100, 6600)
}

# Find indices corresponding to baseline period
baseline_start_idx = int(periods["Baseline"][0] * sfreq / 1000)
baseline_end_idx = int(periods["Baseline"][1] * sfreq / 1000)

# Calculate baseline mean and standard deviation for each electrode
baseline_means = np.mean(data[:, baseline_start_idx:baseline_end_idx], axis=1)
baseline_stds = np.std(data[:, baseline_start_idx:baseline_end_idx], axis=1)

# Group electrodes into batches (10-15 electrodes per plot)
electrode_groups = [ch_names[i:i+15] for i in range(0, len(ch_names), 15)]

def plot_in_new_window(group, picks):
    root = tk.Tk()
    root.title(f'SEEG Data: {", ".join(group)}')
    
    fig, axs = plt.subplots(len(picks), 1, figsize=(15, 3*len(picks)), sharex=True)
    if len(picks) == 1:
        axs = [axs]  # Ensure axs is always a list
    
    significant_electrodes = set()  # Set to store significant electrodes
    
    for i, pick in enumerate(picks):
        data, times = raw[pick, :]
        axs[i].plot(times, data.T, label=group[i], color=plt.cm.rainbow(i/len(picks)))
        
        # Mark periods
        for period, (start, end) in periods.items():
            start_prop = start / total_duration
            end_prop = end / total_duration
            axs[i].axvline(x=start_prop * times[-1], color='black', linestyle='--', linewidth=0.5)
            axs[i].axvline(x=end_prop * times[-1], color='black', linestyle='--', linewidth=0.5)
            axs[i].annotate(period, xy=((start_prop + end_prop) / 2 * times[-1], axs[i].get_ylim()[1]), 
                            xycoords='data', ha='center', va='bottom', 
                            bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", lw=1))
        
        # Find and mark the maximum value in each word period if significant
        for period in ["W1", "W2", "W3", "W4"]:
            start, end = periods[period]
            start_prop = start / total_duration
            end_prop = end / total_duration
            start_idx = int(start_prop * len(times))
            end_idx = int(end_prop * len(times))
            period_data = data[:, start_idx:end_idx]
            if period_data.size > 0:  # Ensure the array is not empty
                max_val = np.max(period_data)
                max_time = times[start_idx:end_idx][np.argmax(period_data)]
                
                # Perform t-test between baseline and period data
                t_stat, p_val = ttest_ind(data[:, baseline_start_idx:baseline_end_idx].flatten(), period_data.flatten())
                
                # Annotate if p-value is significant (e.g., p < 0.05)
                if p_val < 0.05:
                    axs[i].plot(max_time, max_val, 'ro')
                    
                    # Compare maximum value with baseline statistics
                    if max_val > baseline_means[pick] + 3 * baseline_stds[pick]:
                        axs[i].annotate(f'Significant\nMax: {max_val:.2f}', (max_time, max_val), 
                                        xytext=(5, 5), textcoords='offset points',
                                        bbox=dict(boxstyle="round,pad=0.3", fc="yellow", ec="black", lw=1))
                        significant_electrodes.add(group[i])  # Add to significant electrodes
        
        axs[i].set_ylabel(group[i])
        axs[i].legend()
    
    axs[-1].set_xlabel('Time (s)')
    axs[-1].set_xlim(0, times[-1])  # Set x-axis limit to the full duration of the data
    plt.tight_layout()
    
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack()
    
    # Create a text widget to display significant electrodes
    text_widget = tk.Text(root, height=5, width=50)
    text_widget.pack()
    text_widget.insert(tk.END, "Significant Electrodes:\n")
    text_widget.insert(tk.END, ", ".join(sorted(significant_electrodes)))
    
    root.mainloop()
    
    return significant_electrodes  # Return the set of significant electrodes

# Plot each group of electrodes and collect significant electrodes
all_significant_electrodes = set()
for group in electrode_groups:
    picks = [info.ch_names.index(ch_name) for ch_name in group]
    significant_electrodes = plot_in_new_window(group, picks)
    all_significant_electrodes.update(significant_electrodes)

# After all windows are closed, print the complete list of significant electrodes
print("All Significant Electrodes:")
print(", ".join(sorted(all_significant_electrodes)))




In [ ]:
subject_language = {}

In [ ]:
subject_language[14] = all_significant_electrodes

In [ ]:
sentence_data.columns

In [ ]:
mod_count = sentence_data['modality'].value_counts()

In [ ]:
mod_count

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size (optional)
mod_count.plot(kind='bar')

# Customize the plot (optional)
plt.title('Histogram of Value Counts')
plt.xlabel('Values')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))  # Set the figure size (optional)
sen_dis.plot(kind='bar')

# Customize the plot (optional)
plt.title('Histogram of Value Counts')
plt.xlabel('Values')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))  # Set the figure size (optional)
elec_count.plot(kind='bar')

# Customize the plot (optional)
plt.title('Histogram of Value Counts')
plt.xlabel('Values')
plt.ylabel('Frequency')

# Show the plot
plt.show()

In [ ]:
total_elec[total_elec['sub_id'] == 2]

In [ ]:
sentence_data


In [ ]:
total_elec.rename(columns = {"type":"sentenceType"}, inplace = True)

In [ ]:
total_elec

In [ ]:
#additional preprocessing and cleaning

In [ ]:
#Dropping flat electrodes
start_col = 0
end_col = 150

# Ensure that end_col does not exceed the number of columns in df
end_col = min(end_col, len(total_elec.columns) - 1)

# Check and drop rows where all values in the specified columns are zero
total_elec = total_elec[(total_elec.iloc[:, start_col:end_col + 1] != 0).any(axis=1)]



In [ ]:
total_elec

In [ ]:
#GLoVe embeddings

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import gensim
import gensim.downloader as api
from scipy.interpolate import interp1d
from sklearn.model_selection import KFold

In [ ]:
#Getting embeddings from GloVe
def load_glove_model(glove_file):
    with open(glove_file, 'r') as f:
        glove_model = {}
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    return glove_model

glove_file = '/Users/klab/Desktop/glove.42B.300d.txt'  # Path to your extracted GloVe file
glove_model = load_glove_model(glove_file)

In [ ]:
#Get rid of "noise" columns not relevant to analysis
sentence_solo_total = sentence_data[['w1','w2','w3','w4','modality','sentenceType', 'sub_id']]
sentence_solo_total.drop_duplicates(subset =['w1','w2','w3','w4','modality','sentenceType'], inplace = True)
sentence_solo_total

In [ ]:
#Need to populate flat lists with each sentence, one index per sentence
sentences = []
#contains the modalilities and sentence types for look-up
lookup_list = []
for i, row in sentence_solo_total.iterrows():
    curr_sen = []
    curr_lookup = []
    #Add each individual word
    curr_sen.append(row['w1'])
    curr_sen.append(row['w2'])
    curr_sen.append(row['w3'])
    curr_sen.append(row['w4'])
    #Append to list 
    sentences.append(curr_sen)
    #add lookup values
    curr_lookup.append(row['modality'])
    curr_lookup.append(row['sentenceType'])
    curr_lookup.append(row['sub_id'])
    lookup_list.append(curr_lookup)

In [ ]:
len(lookup_list)

In [ ]:
len(sentences)

In [ ]:
def get_sentence_embedding(sentence, model):
    embeddings = [model[word] for word in sentence if word in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(300)

# Extract embeddings for each sentence
sentence_embeddings = np.array([get_sentence_embedding(sentence, glove_model) for sentence in sentences_final])

In [ ]:
sentence_embeddings

In [ ]:
#1500 ms between each timestamp 

In [ ]:
len(sentences)

In [ ]:
lookup_df = pd.DataFrame(lookup_list, columns=['modality', 'sentenceType','sub_id'])

In [ ]:
lookup_df.to_csv('lookupdf.csv.gz', compression = 'gzip')

In [ ]:
sentence_indices

In [ ]:
"""
This is the standard ridge regression code to run to compare to GloVe embeddings, BERT embeddings, or GPT embeddings. To choose which 
to compare edit this line filtered_embeddings = gpt_embeddings[sentence_indices] to reflect the name of your desired embeddings.
"""
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Create results DataFrame with the desired columns
results = pd.DataFrame(columns=['Electrode', 'GS_MSE', 'GS_R2', 'GNS_MSE', 'GNS_R2', 'NGNS_MSE', 'NGNS_R2'])

# Group the data by electrode
electrode_groups = elec_final.groupby('elec_source')

# Initialize a scaler
scaler = StandardScaler()

for electrode, electrode_df in electrode_groups:
    result_row = {'Electrode': electrode}
    for condition in ['GS', 'GNS', 'NGNS']:
        # Filter DataFrame by condition for the current electrode
        condition_data = electrode_df[electrode_df['sentenceType'] == condition]
        if len(condition_data) == 0:
            print(f"Warning: No data for {condition} in electrode {electrode}")
            result_row[f'{condition}_MSE'] = np.nan
            result_row[f'{condition}_R2'] = np.nan
            continue
        
        # Filter sentence embeddings based on condition
        condition_lookup = lookup_df[lookup_df['sentenceType'] == condition]
        sentence_indices = condition_lookup.index
        filtered_embeddings = gpt_embeddings[sentence_indices] #Swap out the name of the list here to change to BERT or GloVe
        
        # Extract neural data for the current electrode and condition
        y = condition_data.iloc[:, 2:153].values.flatten()  # Use all 151 timestamps
        
        # Ensure y and filtered_embeddings have the same length
        if len(filtered_embeddings) > len(y):
            filtered_embeddings = filtered_embeddings[:len(y)]
        elif len(y) > len(filtered_embeddings):
            y = y[:len(filtered_embeddings)]
        
        # Standardize the embeddings
        X = scaler.fit_transform(filtered_embeddings)
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Perform Ridge Regression using sklearn
        ridge = Ridge(alpha=2.0)  # Adjust the alpha parameter if necessary
        ridge.fit(X_train, y_train)
        y_pred_train = ridge.predict(X_train)
        y_pred_test = ridge.predict(X_test)
        
        # Calculate Mean Squared Error and R-squared for both training and testing sets
        mse_train = mean_squared_error(y_train, y_pred_train)
        r2_train = r2_score(y_train, y_pred_train)
        mse_test = mean_squared_error(y_test, y_pred_test)
        r2_test = r2_score(y_test, y_pred_test)
        
        # Diagnostics: print mean and variance of y and y_pred
        print(f"Electrode: {electrode}, Condition: {condition}")
        print(f"y_train mean: {np.mean(y_train)}, y_train variance: {np.var(y_train)}")
        print(f"y_pred_train mean: {np.mean(y_pred_train)}, y_pred_train variance: {np.var(y_pred_train)}")
        print(f"Train MSE: {mse_train}, Train R2: {r2_train}")
        print(f"y_test mean: {np.mean(y_test)}, y_test variance: {np.var(y_test)}")
        print(f"y_pred_test mean: {np.mean(y_pred_test)}, y_pred_test variance: {np.var(y_pred_test)}")
        print(f"Test MSE: {mse_test}, Test R2: {r2_test}")
        
        # Store results
        result_row[f'{condition}_MSE'] = mse_test
        result_row[f'{condition}_R2'] = r2_test
    
    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)

print(results)


In [ ]:
gs_results = gs_results[(gs_results['R2'] < 1) & (gs_results['R2'] > -1)]
gns_results = gns_results[(gns_results['R2'] < 1)  & (gns_results['R2'] > -1)]
ngns_results = ngns_results[(ngns_results['R2'] < 1)  & (ngns_results['R2'] > -1)]

In [ ]:
results.dropna(how = 'any', inplace = True)
#GPT has lower error and higher range of p values/r-squared values

In [ ]:
results.to_csv('bert_results_frame.csv.gz', compression = 'gzip')

In [ ]:
subject_14_elec_total['sentenceType'].replace({'NG':'GS'}, inplace = True)
subject_14_elec_total['sentenceType'].value_counts()

In [ ]:
results_r2 = results[['GS_R2','GNS_R2','NGNS_R2']]

In [ ]:
"""
If looking for summary information/stats run this cell
"""
print('number of sentences')
len(sentences)
print('value counts of sentence types and modalities')
print(sentence_solo_total['sentenceType'].value_counts())
print(sentence_solo_total['modality'].value_counts())
print('neural sentence value counts, one row per modality per electrode <- 161 electrodes')
print(elec_final['sentenceType'].value_counts())
print('Subject info')
print(elec_final['sub_id'].value_counts())
print('151 time stamps per electrode per condition')
print('modality info')

In [ ]:
#Plotting stuff 

In [ ]:
# Filter out high errors (adjust threshold as needed)
mse_threshold = 10000  # Adjust this threshold based on your requirements
filtered_results = results[(results['GS_MSE'] < mse_threshold) & (results['GNS_MSE'] < mse_threshold) & (results['NGNS_MSE'] < mse_threshold)]

# Plot histograms for R2 values, 20 electrodes per plot
log_transform_constant = 1e-6
num_electrodes = len(filtered_results)
num_plots = (num_electrodes // 20) + (1 if num_electrodes % 20 else 0)
conditions = ['GS', 'GNS', 'NGNS']
colors = {'GS': 'blue', 'GNS': 'green', 'NGNS': 'red'}

for plot_idx in range(num_plots):
    fig, ax = plt.subplots(figsize=(15, 7))
    start_idx = plot_idx * 20
    end_idx = min(start_idx + 20, num_electrodes)
    electrodes = filtered_results.index[start_idx:end_idx]
    
    bar_width = 0.25
    x = np.arange(len(electrodes))
    
    for i, condition in enumerate(conditions):
        r2_values = filtered_results.loc[electrodes, f'{condition}_R2']
        ax.bar(x + i * bar_width, r2_values + log_transform_constant, bar_width, color=colors[condition], label=condition)
    
    ax.set_title(f'R² Values for Electrodes {start_idx + 1} to {end_idx}')
    ax.set_xlabel('Electrode')
    ax.set_ylabel('R² (log scale)')
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(electrodes)
    ax.set_yscale('log')
    ax.legend()
    plt.show()

In [ ]:
#Now only do significant electrodes
significant_threshold = 0.05
significant_results = filtered_results[
    (filtered_results['GS_p_value'] < significant_threshold) |
    (filtered_results['GNS_p_value'] < significant_threshold) |
    (filtered_results['NGNS_p_value'] < significant_threshold)
]

In [ ]:
def get_significance_asterisks(p_value):
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''

for plot_idx in range(num_plots):
    fig, ax = plt.subplots(figsize=(15, 7))
    start_idx = plot_idx * 20
    end_idx = min(start_idx + 20, num_electrodes)
    electrodes = significant_results.index[start_idx:end_idx]
    
    bar_width = 0.25
    x = np.arange(len(electrodes))
    
    for i, condition in enumerate(conditions):
        r2_values = significant_results.loc[electrodes, f'{condition}_R2']
        p_values = significant_results.loc[electrodes, f'{condition}_p_value']
        bars = ax.bar(x + i * bar_width, r2_values + log_transform_constant, bar_width, color=colors[condition], label=condition)
        
        for bar, p_value in zip(bars, p_values):
            significance = get_significance_asterisks(p_value)
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), significance, ha='center', va='bottom', fontsize=12)
    
    ax.set_title(f'R² Values for Electrodes {start_idx + 1} to {end_idx} (Significant)')
    ax.set_xlabel('Electrode')
    ax.set_ylabel('R² (log scale)')
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(electrodes)
    ax.set_yscale('log')
    ax.legend()
    plt.show()
    #159/161 electrode significant in GS condition but only 1 star p value. high error with GloVe

In [ ]:
"""
GPT-2-XL and BERT embedding extraction functions below can use torch to save embedding lists to avoid repeat operations
"""

In [29]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

In [30]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = GPT2Model.from_pretrained('gpt2-xl')


def get_gpt2_embeddings(sentences, batch_size=32):
    embeddings = []
    tokenizer.pad_token = tokenizer.eos_token
    # Process sentences in batches
    for i in range(0, len(sentences), batch_size):
        print('batch', i+1)
        batch = sentences[i:i + batch_size]
        
        # Tokenize the input sentences
        inputs = inputs = tokenizer(batch, padding=True, return_tensors='pt')
        
        # Forward pass through the model
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract the last hidden states
        last_hidden_states = outputs.last_hidden_state
        
        # Average the token embeddings to get sentence embeddings
        sentence_embeddings = last_hidden_states.mean(dim=1)
        
        embeddings.append(sentence_embeddings)
        
    
    # Concatenate all embeddings
    embeddings = torch.cat(embeddings, dim=0)
    
    return embeddings

In [ ]:
#handle the pad token


In [ ]:
"""
Both BERT and GPT tokenizers require the sentences to be joined
"""
def join_words_to_sentences(list_of_word_lists):
    return [' '.join(words) for words in list_of_word_lists]

In [ ]:
sentences_joined = join_words_to_sentences(sentences)

In [31]:
gpt_embeddings = torch.load('./sentence_embeddings.pt')
#get_gpt2_embeddings(sentences_joined)

In [ ]:
import torch

# Assuming embeddings is the tensor you want to save
torch.save(gpt_embeddings, 'sentence_embeddings.pt')

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(sentences, batch_size=32):
    embeddings = []
    
    # Process sentences in batches
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the input sentences
        inputs = tokenizer(batch, padding=True, return_tensors='pt', truncation=True)
        
        # Forward pass through the model
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Extract the last hidden states
        last_hidden_states = outputs.last_hidden_state
        
        # Average the token embeddings to get sentence embeddings
        sentence_embeddings = last_hidden_states.mean(dim=1)
        
        embeddings.append(sentence_embeddings)
    
    # Concatenate all embeddings
    embeddings = torch.cat(embeddings, dim=0)
    
    return embeddings

# Example usage



In [ ]:
embeddings = get_bert_embeddings(sentences_joined)

In [ ]:
torch.save(embeddings, 'sentence__embeddings_bert.pt')

In [ ]:
bert_embed = embeddings